In [ ]:
import json
from websockets import connect
import websocket
import pandas as pd 

In [ ]:
# !pip install websocket-client

In [ ]:
# !pip install websocket

In [ ]:
# # Replace with your actual API key
# # API_KEY = "YOUR_API_KEY"

# async def get_candlestick_data():
#     uri = "wss://coinswitch.co/api/ws"  # WebSocket endpoint URL
#     async with connect(uri) as websocket:
#         # Subscribe to the channel for the desired currency pair
#         await websocket.send(json.dumps({"setOp": "subscribe", "args": ["spot-btcusdt"]}))

#         while True:
#             try:
#                 data = await websocket.recv()
#                 # Parse JSON data
#                 parsed_data = json.loads(data)

#                 if "k" in parsed_data:  # Check for candlestick data
#                     # Extract relevant information (replace with specific keys you need)
#                     open_price = parsed_data["k"]["o"]
#                     high_price = parsed_data["k"]["h"]
#                     low_price = parsed_data["k"]["l"]
#                     close_price = parsed_data["k"]["c"]
#                     volume = parsed_da["k"]["v"]

#                     # Process the data (e.g., print, store in database)
#                     print(f"Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

#             except Exception as e:
#                 print(f"Error: {e}")
#                 break  # Handle errors and disconnect if needed

# if __name__ == "__main__":
#     import asyncio
#     asyncio.get_event_loop().run_until_complete(get_candlestick_data())


In [ ]:
# import json
# import websocket

# assets = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT']

# assets = [coin.lower() + '@kline_1m' for coin in assets]
# assets = '/'.join(assets)

# def on_message(ws, message):
#     message = json.loads(message)
#     print(message)

# def on_error(ws, error):
#     print(error)

# def on_close(ws):
#     print("### closed ###")

# def on_open(ws):
#     print("### connected ###")

# socket = "wss://stream.binance.com:9443/stream?streams=" + assets
# ws = websocket.WebSocketApp(socket,
#                             on_message=on_message,
#                             on_error=on_error,
#                             on_close=on_close)
# ws.on_open = on_open

# ws.run_forever()


## EMA strategies for Purchase/sell in any stocks

In [2]:
data.shape

(3581, 11)

In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Sample data creation for demonstration purposes
# Replace 'crypto_prices.csv' with your actual file path and ensure it has 'Date', 'Open', 'High', 'Low', and 'Close' columns
# The following lines are for creating a sample dataset if you don't have one
# Comment out or remove these lines if using an actual dataset
dates = pd.date_range(start='2022-01-01', end='2023-01-01', freq='D')
close_prices = np.random.normal(loc=100, scale=10, size=len(dates))
data = pd.DataFrame({'Close': close_prices}, index=dates)
data['Open'] = data['Close'] + np.random.normal(loc=0, scale=2, size=len(dates))
data['High'] = data[['Close', 'Open']].max(axis=1) + np.random.normal(loc=0, scale=2, size=len(dates))
data['Low'] = data[['Close', 'Open']].min(axis=1) - np.random.normal(loc=0, scale=2, size=len(dates))
data['Date'] = data.index
data.to_csv('crypto_prices.csv')

# Load historical price data
data = pd.read_csv('crypto_prices.csv', parse_dates=['Date'], index_col='Date')

# Calculate EMAs
data['EMA_9'] = data['Close'].ewm(span=9, adjust=False).mean()
data['EMA_21'] = data['Close'].ewm(span=21, adjust=False).mean()
data['EMA_55'] = data['Close'].ewm(span=55, adjust=False).mean()

# Generate buy/sell signals
data['Signal'] = 0
data['Signal'] = np.where((data['EMA_9'] > data['EMA_21']) & (data['EMA_21'] > data['EMA_55']), 1, data['Signal'])
data['Signal'] = np.where((data['EMA_9'] < data['EMA_21']) & (data['EMA_21'] < data['EMA_55']), -1, data['Signal'])

# Create buy and sell columns
data['Buy'] = np.where(data['Signal'] == 1, data['Close'], np.nan)
data['Sell'] = np.where(data['Signal'] == -1, data['Close'], np.nan)

# Create candlestick chart using Plotly Express
fig = go.Figure(data=[go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Candlestick'
)])

# Add EMAs
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_9'], mode='lines', name='9-period EMA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_21'], mode='lines', name='21-period EMA', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_55'], mode='lines', name='55-period EMA', line=dict(color='green')))

# Add buy and sell signals
fig.add_trace(go.Scatter(x=data.index, y=data['Buy'], mode='markers', marker=dict(symbol='triangle-up', color='green', size=10), name='Buy Signal'))
fig.add_trace(go.Scatter(x=data.index, y=data['Sell'], mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Sell Signal'))

# Update layout with range slider and selector
fig.update_layout(
    title='Crypto Buy/Sell Signals with Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=1, label='YTD', step='year', stepmode='todate'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

# Show the plot
fig.show()
